# Interdiction de plus courts chemins

MPRO 2018/2018 - RORT

*Guillaume CROGNIER, Guillaume DALLE, Olivier RIGAL*

## 2. Partie heuristique

Importer les packages nécessaires

In [3]:
using Random; Random.seed!(63);
using Statistics

Importer le générateur d'instances et les outils de graphes

In [2]:
include("../instances_io.jl")
include("../graph_tools.jl");

shortest_path_subgraph (generic function with 1 method)

Génération de l'instance

In [5]:
l = 5 # number of lines in the grid-like graph
c = 5 # number of columns in the grid-like graph
k = 10 # number of edges the adversary can penalize
maxc = 5 # range of initial edge cost
maxd = 50 # range of penalized edge cost
sv = generate(l, c, k, maxc, maxd);

Recherche arborescente

In [6]:
function intelligent_arc_ordering(sv::Data)
    # Temporary
    return get_arcs(sv)
end

intelligent_arc_ordering (generic function with 1 method)

In [7]:
function random_child(ordered_arcs, x, k)
    n_arcs = length(ordered_arcs)
    n_fixed = length(x)
    if (k <= 0) | (n_arcs == n_fixed)
        return copy(x)
    end
    
    left_to_penalize = rand(n_fixed+1:n_arcs, k)

    x_completed = copy(x)
    for (a, (u, v)) in enumerate(ordered_arcs[n_fixed+1:n_arcs])
        if a in left_to_penalize
            x_completed[(u, v)] = 1
        else
            x_completed[(u, v)] = 0
        end
    end
    
    return x_completed
    
end

random_child (generic function with 1 method)

In [8]:
function evaluate_subtree(ordered_arcs, x, k, p)
    if (k == 0) | (length(x) == length(ordered_arcs))
        return shortest_path(sv, x)[1]
    else
        children = [random_child(ordered_arcs, x, k) for child in 1:p]
        mean_cost = mean([shortest_path(sv, x_completed)[1] for x_completed in children])
        return mean_cost
    end
end

evaluate_subtree (generic function with 1 method)

In [9]:
function tree_search(sv::Data, k::Int, p::Int)
    ordered_arcs = intelligent_arc_ordering(sv)
    x = Dict{Tuple{Int, Int}, Int}()

    for next_arc in ordered_arcs

        if k == 0
            x[next_arc] = 0
        else
            x[next_arc] = 0
            mean_cost_0 = evaluate_subtree(ordered_arcs, x, k, p)
            x[next_arc] = 1
            mean_cost_1 = evaluate_subtree(ordered_arcs, x, k-1, p)

            if mean_cost_0 > mean_cost_1
                x[next_arc] = 0
            else
                x[next_arc] = 1
                k -= 1
            end
        end
    end
    
    return shortest_path(sv, x)[1], x
end

tree_search (generic function with 1 method)

In [10]:
tree_search(sv, k, 100)

MethodError: MethodError: Cannot `convert` an object of type SimpleWeightedGraph{Int64,Float64} to an object of type SimpleWeightedDiGraph
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at essentials.jl:154
  SimpleWeightedDiGraph(::SimpleWeightedGraph) at /Users/guillaume/.julia/packages/SimpleWeightedGraphs/yUFrc/src/SimpleWeightedGraphs.jl:140